# Content List 

#### 01. Importing Libraries 
#### 02. Importing Datasets 
#### 03. Combining Dataframes 
#### 04. Task

# 01. Importing Libraries

In [2]:
#Import libraries
import pandas as pd 
import numpy as np
import os 

# 02. Importing Datasets 

In [3]:
path = r"C:\Users\maple\OneDrive\Desktop\CareerFoundry\Instacart Basket Analysis"

In [3]:
#Importing checked orders dataset 
df_ords = pd.read_csv(os.path.join(path,'02 Data','Prepared Data', 'orders_checked.csv'), index_col = False)

In [4]:
#Importing order_products_prior dataset 
df_ords_prior = pd.read_csv(os.path.join(path,'02 Data','Original Data', 'order_products_prior.csv'), index_col = False)

In [5]:
#Check the output
df_ords.head()

,Unnamed: 0.1,Unnamed: 0,order_id,user_id,number_of_orders_placed,orders_day_of_week,order_hour_of_day,days_since_prior_order,no_prior_orders
0,0,0,2539329,1,1,2,8,NaN,True
1,1,1,2398795,1,2,3,7,15.0,False
2,2,2,473747,1,3,3,12,21.0,False
3,3,3,2254736,1,4,4,7,29.0,False
4,4,4,431534,1,5,4,15,28.0,False


# 03. Combining Dataframes

In [6]:
#drop uncessary columns from orders_checked dataframe
df_ords = df_ords.drop(columns = ['Unnamed: 0.1','Unnamed: 0'])

In [7]:
df_ords_prior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB


In [8]:
df_ords_prior.shape

(32434489, 4)

In [5]:
# Checking for missing values in prior orders dataset
df_ords_prior.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

In [6]:
# Creating new dataframe to check for duplicate values in prior orders dataset 
df_ords_prior_dups = df_ords_prior[df_ords_prior.duplicated()]

In [7]:
df_ords_prior_dups

,order_id,product_id,add_to_cart_order,reordered


In [8]:
#Check for mixed-type data in your df_ords_prior dataframe.
for col in df_ords_prior.columns.tolist():
  weird = (df_ords_prior[[col]].applymap(type) != df_ords_prior[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords_prior[weird]) > 0:
    print (col)

In [9]:
# Change types for ords data set

df_ords['order_id']=df_ords['order_id'].astype('int32')
df_ords['user_id'] = df_ords['user_id'].astype('int32')
df_ords['number_of_orders_placed']=df_ords['number_of_orders_placed'].astype('int8')
df_ords['orders_day_of_week']=df_ords['orders_day_of_week'].astype('int8')
df_ords['order_hour_of_day']=df_ords['order_hour_of_day'].astype('int8')
df_ords['days_since_prior_order']=df_ords['days_since_prior_order'].astype('float16')
df_ords['no_prior_orders']=df_ords['no_prior_orders'].astype('object')

In [10]:
# Change types for ords prods data set 

df_ords_prior['product_id'] =df_ords_prior['product_id'].astype('int32')
df_ords_prior['reordered']=df_ords_prior['reordered'].astype('int8')
df_ords_prior['add_to_cart_order']=df_ords_prior['add_to_cart_order'].astype('int32')
df_ords_prior['order_id']=df_ords_prior['order_id'].astype('int32')

In [11]:
#attempt a merge
df_merged_large = df_ords.merge(df_ords_prior, on = ['order_id'], how = 'inner', indicator = True)

In [12]:
df_merged_large.head()

,order_id,user_id,number_of_orders_placed,orders_day_of_week,order_hour_of_day,days_since_prior_order,no_prior_orders,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,True,196,1,0,both
1,2539329,1,1,2,8,NaN,True,14084,2,0,both
2,2539329,1,1,2,8,NaN,True,12427,3,0,both
3,2539329,1,1,2,8,NaN,True,26088,4,0,both
4,2539329,1,1,2,8,NaN,True,26405,5,0,both


In [13]:
#Checking for a full match between dataframes
df_merged_large['_merge'].value_counts()

both          32434489
left_only            0
right_only           0
Name: _merge, dtype: int64

In [15]:
df_merged_large.shape

(32434489, 11)

# 04. Task 

In [14]:
#Export the merged file in pickle format as “orders_products_combined.pkl”.
df_merged_large.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined.pkl'))